In [1]:
import numpy as np
import pyblp
import pandas as pd
pyblp.options.digits = 3
#pyblp.options.verbose = True
np.set_printoptions(precision=3, threshold=10)

from tab_diversion import relative_error, compute_rel_table, compute_avg_table

In [2]:
nevo_products = np.recfromcsv(pyblp.data.NEVO_PRODUCTS_LOCATION)
nevo_agents = np.recfromcsv(pyblp.data.NEVO_AGENTS_LOCATION)
nevo_agent_formulation = pyblp.Formulation('0 + income + income_squared + age + child')

nevo_products = {n: nevo_products[n] for n in nevo_products.dtype.names}
nevo_product_formulations = (
        pyblp.Formulation('0 + prices', absorb='C(product_ids)'),
        pyblp.Formulation('prices + sugar + mushy')
)

# Define the problem
nevo_problem = pyblp.Problem(
    nevo_product_formulations,
    nevo_products,
    nevo_agent_formulation,
    nevo_agents
 )

nevo_problem_logit = pyblp.Problem(
    nevo_product_formulations[0],
    nevo_products,
    None,
    None
 )

nevo_sigma = np.diag([0.3302, 2.4526, 0.0163, 0.2441])

nevo_pi = [[ 5.4819,  0,      0.2037,  0     ],
 [15.8935, -1.2000, 0,       2.6342],
 [-0.2506,  0,      0.0511,  0     ],
 [ 1.2650,  0,     -0.8091,  0     ]
 ]

logit_results=nevo_problem_logit.solve()
nevo_results = nevo_problem.solve( nevo_sigma,nevo_pi, steps=1, optimization=pyblp.Optimization('bfgs'))


Absorbing demand-side fixed effects ...
Absorbed demand-side fixed effects after 0:00:00.

Absorbing demand-side fixed effects ...
Absorbed demand-side fixed effects after 0:00:00.

Computing results for step 1 ...
Computed results after 0:00:00.

Results Summary:
GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Optimization  Total Step  Objective    Gradient   
Step   Iterations   Evaluations     Iterations         Evaluations         Time         Time       Value    Infinity Norm
----  ------------  -----------  -----------------  -----------------  ------------  ----------  ---------  -------------
 1         0             1               0                  0            0:00:00      0:00:00    +1.90E+02       NA      

Linear Estimates (Robust SEs in Parentheses):
Beta:    prices   
-----  -----------
        -3.01E+01 
       (+1.02E+00)

Computing results for step 2 ...
Computed results after 0:00:00.

Results Summary:
GMM   Optimization   Objective   Total 

 1         18           25           547         1724      +1.49E+01   +8.70E-03     +1.18E+00     +6.15E-01 
 1         19           26           503         1590      +1.49E+01   +2.30E-03     +3.67E-01     +8.17E-01 
 1         20           27           506         1600      +1.49E+01   +3.97E-03     +1.01E+00               
 1         21           28           501         1587      +1.49E+01   +7.14E-03     +2.45E+00               
 1         22           29           503         1580      +1.49E+01   +1.31E-02     +4.25E+00               
 1         23           30           506         1587      +1.48E+01   +2.42E-02     +6.63E+00               
 1         24           31           521         1638      +1.48E+01   +4.44E-02     +9.92E+00               
 1         25           32           545         1710      +1.47E+01   +8.03E-02     +1.46E+01               
 1         26           33           577         1824      +1.46E+01   +1.42E-01     +2.11E+01               
 1        

In [3]:
# calculate diversion ratios
mte_diversion=nevo_results.compute_diversion_ratios()
ate_diversion=nevo_results.compute_long_run_diversion_ratios()
# logit has MTE=ATE
logit_diversion=logit_results.compute_long_run_diversion_ratios()

og_mask=np.vstack([np.eye(24).astype(bool) for i in range(0,94)])
table2=compute_rel_table(mte_diversion,ate_diversion,logit_diversion,og_mask)
table1=compute_avg_table(mte_diversion,ate_diversion,logit_diversion,og_mask)

Computing diversion ratios with respect to prices ...
Finished after 0:00:00.

Computing long run mean diversion ratios ...
Finished after 0:00:00.

Computing long run mean diversion ratios ...
Finished after 0:00:00.



In [4]:
table1.round(2)

,MTE,ATE,Logit
Med($D_{jk}$),12.74,13.03,8.89
Mean($D_{jk}$),14.33,14.74,9.84
\% Correct,100.00,89.54,58.82
Med($D_{j0}$),35.06,32.06,54.43
Mean($D_{j0}$),36.58,33.32,53.46


In [5]:
table2.round(2)

,med($y-x$),mean($y-x$),med($|y-x|$),mean($|y-x|$),std($|y-x|$)
$ATE$,2.58,3.16,6.05,7.70,7.01
$Logit$,-41.85,-40.06,43.12,45.58,27.91
$ATE$,6.27,8.67,8.67,12.37,11.98
$Logit$,-33.96,-24.10,48.06,52.01,34.14
$ATE$,-8.40,-9.41,8.42,9.58,7.01
$Logit$,40.37,40.10,40.39,41.45,22.40
